In [ ]:
import pandas as pd
import numpy as np
import json

!pip install pyLDAvis
!pip install -U pandas
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import spacy
import string
from tqdm import tqdm
tqdm.pandas()

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

stops = set(stopwords.words('english'))
stops.add('-pron-')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,movieId,title_x,genres,year,user_tag_list,imdb_id,tmdbId,title_y,overview,production_companies,production_countries,spoken_languages,tagline
0,0.0,1.0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",['1995'],"['Owned', 'imdb top 250', 'Pixar', 'Pixar', 't...",114709.0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",['Pixar Animation Studios'],NaN,NaN,NaN
1,1.0,2.0,Jumanji,"[Adventure, Children, Fantasy]",['1995'],"['Robin Williams', 'time travel', 'fantasy', ""...",113497.0,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,"['TriStar Pictures', 'Teitler Film', 'Intersco...",NaN,NaN,Roll the dice and unleash the excitement!


In [ ]:
# Load csv
df_original = pd.read_csv('content_df.csv')
df_original = df_original[~(df_original['title_x'].isna() | df_original['overview'].isna() | df_original['overview'] == 'No overview found.')]

df_original = df_original[~df_original['overview'].isna()]
df_original = df_original[~df_original['title_x'].isna()]
df_original = df_original[~df_original['overview'].apply(lambda x: 'no overview' in x.lower())]

df = df_original.copy()

df.loc[df['genres'].isin([' no genres listed ', np.NaN])] = np.NaN

# Split genre string into list of genres
df['genres'] = df['genres'].str.split(' ')
df.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,movieId,title_x,genres,year,user_tag_list,imdb_id,tmdbId,title_y,overview,production_companies,production_countries,spoken_languages,tagline
0,0.0,1.0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",['1995'],"['Owned', 'imdb top 250', 'Pixar', 'Pixar', 't...",114709.0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",['Pixar Animation Studios'],NaN,NaN,NaN
1,1.0,2.0,Jumanji,"[Adventure, Children, Fantasy]",['1995'],"['Robin Williams', 'time travel', 'fantasy', ""...",113497.0,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,"['TriStar Pictures', 'Teitler Film', 'Intersco...",NaN,NaN,Roll the dice and unleash the excitement!


In [ ]:
# Only include rows with at least one genre
df = df[~df['genres'].isna()]

# Get list of genres
genres = {}
for row in df['genres']:
    for g in row:
        if g not in genres:
            genres[g] = 1
        else:
            genres[g] += 1

# Sort genres by count
genres = sorted(genres.items(), key=lambda item: item[1], reverse = True)

# Only use genres with at least 1000 results as main genres
main_genres = {k: v for k, v in genres if v >= 1000}
main_genres

{'Action': 3594,
 'Adventure': 2318,
 'Animation': 1020,
 'Children': 1360,
 'Comedy': 8244,
 'Crime': 2979,
 'Documentary': 1469,
 'Drama': 12973,
 'Fantasy': 1449,
 'Fi': 1771,
 'Horror': 2633,
 'Mystery': 1488,
 'Romance': 4236,
 'Sci': 1771,
 'Thriller': 3925,
 'War': 1167}

In [ ]:
# Only include rows with at least one main genre
def is_main_genre(g_dict):
    for g in g_dict:
        if g in main_genres:
            return True
    return False

df = df[df['genres'].map(is_main_genre)]
print('Remaining rows:', df.shape[0])

Remaining rows: 26031


In [ ]:
# Load spacy nlp
nlp = spacy.load('en')
vocab = set()

# Remove named entities and non-alpha tokens
def preprocess(s):
    s.translate(str.maketrans('', '', string.punctuation))
    document = nlp(s)

    text_no_namedentities = []
    ents = [e.text for e in document.ents]
    for item in document:
        word = item.text.lower
        if (word not in ents) and (item.is_alpha) and (word not in stops):
            lemma = item.lemma_.lower()
            vocab.add(item.lemma_)
            text_no_namedentities.append(lemma)
    return text_no_namedentities

# Remove NaN and clean (this part takes ~10 min on google colab)
df = df[df['overview'].notna()]
df['overview_clean'] = df['overview'].progress_map(preprocess)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 26031/26031 [07:48<00:00, 55.55it/s]


In [ ]:
# Remove stopwords
stops = set(stopwords.words('english'))
stops.add('-pron-')
df['overview_clean'] = df['overview_clean'].apply(lambda x: [i for i in x if i not in stops])

In [ ]:
df.to_csv('content_df_clean.csv')
# df = pd.read_csv('content_df_clean.csv')

In [ ]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaModel

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

genre_models = {g: {} for g in main_genres}

for g in tqdm(main_genres):

    df_subset = df[df['genres'].apply(lambda x: g in x)]

    genre_models[g]['dictionary'] = Dictionary(df_subset['overview_clean'])
    genre_models[g]['corpus'] = [genre_models[g]['dictionary'].doc2bow(line) for line in df_subset['overview_clean']]

    genre_models[g]['tfidf'] = TfidfModel(genre_models[g]['corpus'])
    genre_models[g]['tfidf_corpus'] = genre_models[g]['tfidf'][genre_models[g]['corpus']]

    genre_models[g]['lda'] = LdaModel(genre_models[g]['tfidf_corpus'], num_topics=4)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████| 16/16 [00:55<00:00,  3.47s/it]


In [ ]:
pyLDAvis.enable_notebook()

genre = 'Action'

vis_data = gensimvis.prepare(genre_models[genre]['lda'], genre_models[genre]['corpus'], genre_models[genre]['dictionary'])
pyLDAvis.display(vis_data)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
dictionary = Dictionary(df['overview_clean'])
corpus = [genre_models[g]['dictionary'].doc2bow(line) for line in df['overview_clean']]

for g in tqdm(main_genres):

    tfidf = genre_models[g]['tfidf']
    doc = tfidf[corpus]
    vectors = genre_models[g]['lda'][doc]

    columns = np.array([[x[1] for x in v] for v in vectors]).transpose()

    for i in range(4):
        df[g + str(i)] = columns[i]

df.head(2)

100%|██████████| 16/16 [04:00<00:00, 15.03s/it]


NameError: ignored

In [ ]:
for g in main_genres:
    is_genre = df['genres'].apply(lambda x: 1 if g in x else 0)
    for i in range(4):
        df[g + str(i)] = df[g + str(i)] * is_genre

df.head(2)

,Unnamed: 0,movieId,title_x,genres,year,user_tag_list,imdb_id,tmdbId,title_y,overview,production_companies,production_countries,spoken_languages,tagline,overview_clean,Drama0,Drama1,Drama2,Drama3,Comedy0,Comedy1,Comedy2,Comedy3,Romance0,Romance1,Romance2,Romance3,Thriller0,Thriller1,Thriller2,Thriller3,Action0,Action1,Action2,Action3,Crime0,Crime1,Crime2,Crime3,Horror0,Horror1,Horror2,Horror3,Adventure0,Adventure1,Adventure2,Adventure3,Sci0,Sci1,Sci2,Sci3,Fi0,Fi1,Fi2,Fi3,Mystery0,Mystery1,Mystery2,Mystery3,Documentary0,Documentary1,Documentary2,Documentary3,Fantasy0,Fantasy1,Fantasy2,Fantasy3,Children0,Children1,Children2,Children3,War0,War1,War2,War3,Animation0,Animation1,Animation2,Animation3
0,0.0,1.0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",['1995'],"['Owned', 'imdb top 250', 'Pixar', 'Pixar', 't...",114709.0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",['Pixar Animation Studios'],NaN,NaN,NaN,"[lead, woody, andy, toy, live, happily, room, ...",0.0,0.0,0.0,0.0,0.497278,0.402042,0.05131,0.04937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833247,0.066475,0.049374,0.050904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.847380,0.050870,0.050446,0.051304,0.847251,0.051269,0.050495,0.050986,0.0,0.0,0.0,0.0,0.048533,0.050455,0.851751,0.04926
1,1.0,2.0,Jumanji,"[Adventure, Children, Fantasy]",['1995'],"['Robin Williams', 'time travel', 'fantasy', ""...",113497.0,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,"['TriStar Pictures', 'Teitler Film', 'Intersco...",NaN,NaN,Roll the dice and unleash the excitement!,"[sibling, judy, peter, discover, enchanted, bo...",0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.410072,0.466852,0.051150,0.071926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.252473,0.049105,0.650524,0.047898,0.617610,0.053673,0.280690,0.048026,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000


In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
df.to_csv('content_genre_lda.csv')